#### Integrin

## setup

In [1]:
import os
import sys
import math
import time
import pickle
import copy
#import mdtraj
import MDAnalysis as mda
#import pyemma
#import pyemma.util.contexts
import networkx as nx
import random
#import parmed as pmd
#import acpype
import pandas as pd
from MDAnalysis.analysis import distances
from MDAnalysis.analysis import rms


import numpy as np
from scipy.linalg import eigh
from scipy.spatial.distance import pdist
from scipy.interpolate import interp1d
import matplotlib as mpl
from matplotlib import ticker
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mpl.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


import nglview as nv

import glob

no display found. Using non-interactive Agg backend


In [2]:

from importlib import reload 
import sys


#### plot settings

In [3]:
showPlots=1
useMagics=1
if useMagics:
    %matplotlib inline
    #%matplotlib notebook
    %load_ext autoreload
    %autoreload 2
    %reload_ext autoreload
    
font = {'weight' : 'normal',
        'size'   : 25}

plt.rc('font', **font)

## Create MD analysis universe

In [4]:

# Domain composition alpha IIb:
# beta propeller: 1-452
# Thigh: 453-608
# Calf-1: 609-743
# Calf-2: 744-964
# Transmembrane: 965-1008

# Domain composition  Beta 3:
# beta I: residue 110-352
# Hyb: residue 58-109 + 353-432
# Psi: residue 1-57
# E1: 433-475
# E2: 476-525
# E3: 526-560
# E4: 561-604
# beta-T: 605-690
# Transmembrane: 691-772


# final_string_universe_image = {}

# for images in range(19):

#     final_string_struct = path_to_structures+'/iter147/target_md_'+str(images)+'/minimize-protein.pdb'
#     final_string_universe = mda.Universe(final_string_struct) 
#     final_string_universe_image[images] = final_string_universe
            
   

In [51]:
from MDAnalysis import transformations
from MDAnalysis.analysis import align

path_to_structures = '/project2/andrewferguson/sivadasetty/doe/analysis-integrin/string_mechanisms/deadbolt/string_parallel_100k_full_lipid_case/final_string/'
# path_to_structures='/project2/andrewferguson/sivadasetty/doe/analysis-integrin/string_mechanisms/deadbolt/string_parallel_100k_full_lipid_case/string/'


hexcolors = [
    "#1f77b4",  # blue
    "#aec7e8",  # light blue
    "#ff7f0e",  # orange
    "#ffbb78",  # light orange
    "#2ca02c",  # green
    "#98df8a",  # light green
    "#d62728",  # red [hyb1]
    "#d62728",  # red [hyb2]
    "#ff9896",  # light red
    "#9467bd",  # purple
    "#c5b0d5",  # light purple
    "#8c564b",  # brown
    "#c49c94",  # light brown
    "#e377c2",  # pink
    "#f7b6d2",  # light pink
    "#7f7f7f",  # gray
    "#c7c7c7",  # light gray
    "#bcbd22",  # olive
    "#dbdb8d",  # light olive
    "#17becf",  # teal
    "#9edae5"   # light teal
]

views_final_string = {}

for string_image in [8]:
    
    view = nv.NGLWidget()
    view.clear()
    

#     final_string_struct = path_to_structures+'/iter147/target_md_'+str(string_image)+'/minimize-protein.pdb'
    
    final_string_struct = path_to_structures+'/iter2/target_md_'+str(string_image)+'/minimize-protein.pdb'
#     final_string_struct = path_to_structures+'/iter249/target_md_'+str(string_image)+'/minimize-protein.pdb'
    
    if os.path.exists(final_string_struct):
        print('file exists')
    else:
        break
    
    print(final_string_struct)
    
    final_string_universe = mda.Universe(final_string_struct) 

    
    rotation = transformations.rotate.rotateby(90, direction=[0,0,-1], ag=final_string_universe.atoms)
    rotation2 = transformations.rotate.rotateby(-90, direction=[1,0,0], ag=final_string_universe.atoms)
    rotation3 = transformations.rotate.rotateby(180, direction=[0,1,0], ag=final_string_universe.atoms)
#     rotation3 = transformations.rotate.rotateby(270, direction=[0,1,0], ag=final_string_universe.atoms)
    
    
    final_string_universe.trajectory.add_transformations(rotation,rotation2,rotation3)
    
    final_string_universe_beta_propeller = final_string_universe.select_atoms('segid is A').atoms.select_atoms('resid 1 to 452')
    final_string_universe_thigh = final_string_universe.select_atoms('segid is A').atoms.select_atoms('resid 452 to 608')
    final_string_universe_calf1 = final_string_universe.select_atoms('segid is A').atoms.select_atoms('resid 608 to 743')
    final_string_universe_calf2 = final_string_universe.select_atoms('segid is A').atoms.select_atoms('resid 743 to 964')
    final_string_universe_alpha_tm = final_string_universe.select_atoms('segid is A').atoms.select_atoms('resid 964 to 1008')

    final_string_universe_beta_I = final_string_universe.select_atoms('segid is B').atoms.select_atoms('resid 109 to 352')
    final_string_universe_hyb1 = final_string_universe.select_atoms('segid is B').atoms.select_atoms('resid 57 to 109')
    final_string_universe_hyb2 = final_string_universe.select_atoms('segid is B').atoms.select_atoms('resid 352 to 432')
    final_string_universe_psi = final_string_universe.select_atoms('segid is B').atoms.select_atoms('resid 1 to 57')
    final_string_universe_e1 = final_string_universe.select_atoms('segid is B').atoms.select_atoms('resid 432 to 475')
    final_string_universe_e2 = final_string_universe.select_atoms('segid is B').atoms.select_atoms('resid 475 to 525')
    final_string_universe_e3 = final_string_universe.select_atoms('segid is B').atoms.select_atoms('resid 525 to 560')
    final_string_universe_e4 = final_string_universe.select_atoms('segid is B').atoms.select_atoms('resid 560 to 604')
    final_string_universe_beta_t = final_string_universe.select_atoms('segid is B').atoms.select_atoms('resid 604 to 690')
    final_string_universe_beta_tm = final_string_universe.select_atoms('segid is B').atoms.select_atoms('resid 690 to 772')
        
    all_alpha_string_domains = [final_string_universe_beta_propeller, final_string_universe_thigh, 
                                final_string_universe_calf1, final_string_universe_calf2, 
                                final_string_universe_alpha_tm]

    all_beta_string_domains = [final_string_universe_beta_I, final_string_universe_hyb1,
                               final_string_universe_hyb2, final_string_universe_psi, 
                               final_string_universe_e1, final_string_universe_e2,
                               final_string_universe_e3, final_string_universe_e4, 
                               final_string_universe_beta_t, final_string_universe_beta_tm]

    count_i = 0
    count_colors = 0
    for e in all_alpha_string_domains:

        if e.atoms.n_atoms != 0:
            view.add_trajectory(nv.MDAnalysisTrajectory(e))
            view.clear_representations(component=count_i)
            #print(count_colors, count_i, e.atoms.n_atoms, hexcolors[count_colors])
            view.add_representation('cartoon', selection='all', radiusScale=1, component=count_i, 
                                    colorScheme='uniform', color=hexcolors[count_colors])                                
            count_i+=1
        count_colors+=1
        
    for e in all_beta_string_domains:

        if e.atoms.n_atoms != 0:
            #print(e.atoms.n_atoms)
            #print(count_colors, count_i, e.atoms.n_atoms, hexcolors[count_colors])
            view.add_trajectory(nv.MDAnalysisTrajectory(e))
            view.clear_representations(component=count_i)
            view.add_representation('cartoon', selection='all', radiusScale=1, component=count_i, 
                                    colorScheme='uniform', color=hexcolors[count_colors])
            count_i+=1
        count_colors+=1
            
                
#     view.control.spin([1, 0, 0], math.pi/2)
    view._set_size('800px', '800px')
    view.camera = 'orthographic'
             
    views_final_string[string_image] = view


file exists
/project2/andrewferguson/sivadasetty/doe/analysis-integrin/string_mechanisms/deadbolt/string_parallel_100k_full_lipid_case/final_string//iter2/target_md_8/minimize-protein.pdb


In [52]:

view


NGLWidget()

In [53]:

for string_image, view in views_final_string.items():

    display(view)
    view.download_image(filename='final_integrin_string_'+str(string_image)+'-init.png', factor=10, 
                        antialias=False, trim=True, transparent=True)
#     view.download_image(filename='final_integrin_string_'+str(string_image)+'.png', factor=10, 
#                         antialias=False, trim=True, transparent=True)


#     view.download_image(filename='initial_integrin_string_'+str(string_image)+'-init.png', factor=10, 
#                         antialias=False, trim=True, transparent=True)
#     view.download_image(filename='initial_final_integrin_string_'+str(string_image)+'.png', factor=10, 
#                         antialias=False, trim=True, transparent=True)



NGLWidget(n_components=15)

### 

In [87]:
temp_mduniverse = mda.Universe('/project2/andrewferguson/sivadasetty/doe/analysis-integrin/string_mechanisms/deadbolt/string_parallel_100k_full_lipid_case/string/iter0/target_md_12/minimize.pdb',
                               '/project2/andrewferguson/sivadasetty/doe/analysis-integrin/string_mechanisms/deadbolt/string_parallel_100k_full_lipid_case/string/iter0/target_md_12/output.xtc'
                              )

view = nv.show_mdanalysis(temp_mduniverse)

view


NGLWidget(max_frame=50)